In [1]:
import codecs
import numpy as np
import tensorflow as tf
import pandas as pd 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd "/content/drive/My Drive"

/content/drive/My Drive


In [4]:
dialogs = pd.read_csv('movie_conversations.txt', sep = '\+\+\+\$\+\+\+', header = None, engine='python',)
dialogs.head()

,0,1,2,3
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"
3,u0,u2,m0,"['L204', 'L205', 'L206']"
4,u0,u2,m0,"['L207', 'L208']"


In [5]:
dialogs[3] = dialogs[3].str.replace('[', '')
dialogs[3] = dialogs[3].str.replace(']', '')
dialogs[3] = dialogs[3].str.replace("'", '')
# dialogs[3] = dialogs[3].str.replace("L", '')
dialogs[3] = dialogs[3].str.strip()
# dialogs[3].apply()
dialogs.head()

,0,1,2,3
0,u0,u2,m0,"L194, L195, L196, L197"
1,u0,u2,m0,"L198, L199"
2,u0,u2,m0,"L200, L201, L202, L203"
3,u0,u2,m0,"L204, L205, L206"
4,u0,u2,m0,"L207, L208"


In [6]:
test = dialogs[3][301].split(sep = ', ')
test

['L3496', 'L3497']

In [7]:
replies = pd.read_csv('movie_lines.txt', sep = '\+\+\+\$\+\+\+', header = None, engine='python',index_col=0)#
replies.head()

,1,2,3,4
0,,,,
L1045,u0,m0,BIANCA,They do not!
L1044,u2,m0,CAMERON,They do to!
L985,u0,m0,BIANCA,I hope so.
L984,u2,m0,CAMERON,She okay?
L925,u0,m0,BIANCA,Let's go.


In [8]:
replies.index = replies.index.str.strip()
replies.index

Index(['L1045', 'L1044', 'L985', 'L984', 'L925', 'L924', 'L872', 'L871',
       'L870', 'L869',
       ...
       'L666262', 'L666522', 'L666521', 'L666520', 'L666372', 'L666371',
       'L666370', 'L666369', 'L666257', 'L666256'],
      dtype='object', name=0, length=304713)

In [9]:
train = []
labels = []
def make_lists(dialogs_str):
  replies_num = dialogs_str.split(sep = ', ')
  count = (len(replies_num))-1
  for i in range(count):
    train.append(str(replies.loc[[replies_num[i]]][4].values[0]).strip())
    labels.append(str(replies.loc[[replies_num[i+1]]][4].values[0]).strip())

dialogs[3][:10000].apply(make_lists)
# dialogs[3].apply(make_lists)

0       None
1       None
2       None
3       None
4       None
        ... 
9995    None
9996    None
9997    None
9998    None
9999    None
Name: 3, Length: 10000, dtype: object

In [10]:
len(train)

25149

In [11]:
def prepare_vocab(texts):
    vocab = sorted(set(''.join(texts)))
    vocab.append('<START>')
    vocab.append('<END>')
    vocab_size = len(vocab)
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return vocab_size, char2idx, idx2char

INPUT_VOCAB_SIZE, input_char2idx, input_idx2char = prepare_vocab(train)
TARGET_VOCAB_SIZE, target_char2idx, target_idx2char = prepare_vocab(labels)

In [12]:
input_texts_as_int = [[input_char2idx[c] for c in text] for text in train]
target_texts_as_int = [[target_char2idx[c] for c in text] for text in labels]

encoder_input_seqs = [np.array(text) for text in input_texts_as_int]
decoder_input_seqs = []
decoder_target_seqs = []
for target_text in target_texts_as_int:
    decoder_input_seqs.append(np.array([target_char2idx['<START>']] + target_text))
    decoder_target_seqs.append(np.array(target_text + [target_char2idx['<END>']]))

In [13]:
encoder_input_seqs

[array([32, 59, 72,  1, 81, 63,  1, 71, 59, 69, 63,  1, 78, 66, 67, 77,  1,
        75, 79, 67, 61, 69, 29,  1,  1, 47, 73, 82, 59, 72, 72, 63,  1, 40,
        73, 76, 76, 67, 72, 63,  1, 59, 72, 62,  1, 30, 72, 62, 76, 63, 81,
         1, 31, 59, 76, 76, 63, 78, 78,  1, 59, 76, 63,  1, 66, 59, 80, 67,
        72, 65,  1, 59, 72,  1, 67, 72, 61, 76, 63, 62, 67, 60, 70, 83,  1,
        66, 73, 76, 76, 63, 72, 62, 73, 79, 77,  1, 74, 79, 60, 70, 67, 61,
         1, 60, 76, 63, 59, 69, 12,  1, 79, 74,  1, 73, 72,  1, 78, 66, 63,
         1, 75, 79, 59, 62, 13,  1,  1, 30, 65, 59, 67, 72, 13]),
 array([52, 63, 70, 70, 11,  1, 38,  1, 78, 66, 73, 79, 65, 66, 78,  1, 81,
        63,  8, 62,  1, 77, 78, 59, 76, 78,  1, 81, 67, 78, 66,  1, 74, 76,
        73, 72, 79, 72, 61, 67, 59, 78, 67, 73, 72, 11,  1, 67, 64,  1, 78,
        66, 59, 78,  8, 77,  1, 73, 69, 59, 83,  1, 81, 67, 78, 66,  1, 83,
        73, 79, 13]),
 array([43, 73, 78,  1, 78, 66, 63,  1, 66, 59, 61, 69, 67, 72, 65,  1, 59,


In [14]:
# max_enc_seq_length = max([len(seq) for seq in encoder_input_seqs])
# max_dec_seq_length = max([len(seq) for seq in decoder_input_seqs])

max_enc_seq_length = 256
max_dec_seq_length = 256


encoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    encoder_input_seqs,
    value=input_char2idx[' '],
    padding='post',
    maxlen=max_enc_seq_length)

decoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_input_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

decoder_target_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_target_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

In [15]:
decoder_target_seqs

array([[53, 64, 71, ...,  1,  1,  1],
       [44, 74, 79, ...,  1,  1,  1],
       [45, 70, 60, ...,  1,  1,  1],
       ...,
       [32, 80, 79, ...,  1,  1,  1],
       [39, 79,  8, ...,  1,  1,  1],
       [40, 64, 78, ...,  1,  1,  1]], dtype=int32)

In [16]:
H_SIZE = 256 # Размерность скрытого состояния LSTM
EMB_SIZE = 256 # размерность эмбеддингов (и для входных и для выходных цепочек)

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(INPUT_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.lstm1 = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)
        
    def call(self, x):
        out = self.embed(x)
        out, h, c = self.lstm(out)
        _, h1, c1 = self.lstm1(out)
        state = (h, c)
        state1 = (h1, c1)
        return (state, state1)

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(TARGET_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.lstm1 = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(TARGET_VOCAB_SIZE, activation='softmax')
        
    def call(self, x, init_state):
        out = self.embed(x)
        out, h, c = self.lstm(out, initial_state=init_state[0])
        out, h1, c1 = self.lstm1(out, initial_state=init_state[1])
        out = self.fc(out)
        state = (h, c)
        state1 = (h1, c1)
        return out, (state, state1)

encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
def seq2seq_inference(input_seq):
    state = encoder_model(input_seq)

    target_seq = np.array([[target_char2idx['<START>']]])

    decoded_sentence = ''
    while True:
        output_tokens, state = decoder_model(target_seq, state)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_idx2char[sampled_token_index]
        if (sampled_char == '<END>' or
          len(decoded_sentence) > max_dec_seq_length):
           break
        else:
          decoded_sentence += sampled_char


        target_seq = np.array([[sampled_token_index]])

    return decoded_sentence

In [20]:
def ask_my_bot(reply):
  test = [[input_char2idx[c] for c in reply]]
  test = [np.array(text) for text in test]
  test = tf.keras.preprocessing.sequence.pad_sequences(
      test,
      value=input_char2idx[' '],
      padding='post',
      maxlen=max_enc_seq_length)

  decoded_sentence = seq2seq_inference(test)
  print('-',reply)
  print('-',decoded_sentence)
ask_my_bot('Hello!')

- Hello!
- He's alive.


In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
loss = tf.losses.SparseCategoricalCrossentropy()
seq2seq.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
# seq2seq.build()
seq2seq.load_weights('weights7')

ask_my_bot('Hello!')
ask_my_bot('How are you?')
ask_my_bot('You know French?')
ask_my_bot('Bye')



for i in range(20):

  seq2seq.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs,
            # steps_per_epoch=300,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS)
  
  ask_my_bot('Hello!')
  ask_my_bot('How are you?')
  ask_my_bot('You know French?')
  ask_my_bot('Bye')

  seq2seq.save_weights('weights'+str(i))

In [ ]:
    test = 159
    input_seq = encoder_input_seqs[test:test+1]
    decoded_sentence = seq2seq_inference(input_seq)
    print('-')
    print('Input sentence:', train[test])
    print('Result sentence:', decoded_sentence)
    print('Target sentence:', labels[test])